In [1]:
import pandas as pd
import numpy as np

In [2]:
data=pd.read_csv("./data/train.csv")
data

,id,carat,cut,color,clarity,depth,table,x,y,z,price
0,0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453
...,...,...,...,...,...,...,...,...,...,...,...
193568,193568,0.31,Ideal,D,VVS2,61.1,56.0,4.35,4.39,2.67,1130
193569,193569,0.70,Premium,G,VVS2,60.3,58.0,5.75,5.77,3.47,2874
193570,193570,0.73,Very Good,F,SI1,63.1,57.0,5.72,5.75,3.62,3036
193571,193571,0.34,Very Good,D,SI1,62.9,55.0,4.45,4.49,2.81,681


In [3]:
data.drop(['id'],axis=1,inplace=True)
data

,carat,cut,color,clarity,depth,table,x,y,z,price
0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453
...,...,...,...,...,...,...,...,...,...,...
193568,0.31,Ideal,D,VVS2,61.1,56.0,4.35,4.39,2.67,1130
193569,0.70,Premium,G,VVS2,60.3,58.0,5.75,5.77,3.47,2874
193570,0.73,Very Good,F,SI1,63.1,57.0,5.72,5.75,3.62,3036
193571,0.34,Very Good,D,SI1,62.9,55.0,4.45,4.49,2.81,681


In [4]:
# create independent and dependent variable
X=data.drop(['price'],axis=1)
y=data['price']

In [5]:
cat_col=X.select_dtypes(include='object').columns

In [6]:
num_col=X.select_dtypes(exclude='object').columns

In [7]:
from sklearn.impute import SimpleImputer # --> To Handling Missing values
from sklearn.preprocessing import StandardScaler # --> To Handling Feature Scaling
from sklearn.preprocessing import OrdinalEncoder # --> To Perform Encoding
# ---- To Perform Pipeline ------
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer # --> New to meeee

In [8]:
num_pipeline=Pipeline(

    steps=[
        ('imputer',SimpleImputer()),
        ('scaler',StandardScaler())
    ]

)

In [9]:
cat_col

Index(['cut', 'color', 'clarity'], dtype='object')

In [8]:
cut_cat=X[cat_col[0]].unique()
color_cat=X[cat_col[1]].unique()
clarity_cat=X[cat_col[2]].unique()

In [11]:
clarity_cat

array(['VS2', 'SI2', 'VS1', 'SI1', 'IF', 'VVS2', 'VVS1', 'I1'],
      dtype=object)

In [11]:
cat_pipeline=Pipeline(

    steps=[
        ('imputer',SimpleImputer(strategy='most_frequent')),
        ('ordinalencode',OrdinalEncoder(categories=[cut_cat,color_cat,clarity_cat]))
    ]
)

In [12]:
num_col

Index(['carat', 'depth', 'table', 'x', 'y', 'z'], dtype='object')

In [13]:
preprocessor=ColumnTransformer(
    [
        ('num_pipeline',num_pipeline,num_col),
        ('cat_pipeline',cat_pipeline,cat_col)
    ]
)

In [14]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=30)

In [15]:
# do preprocess using pipelie
X_train=pd.DataFrame(preprocessor.(X_train),columns=preprocessor.get_feature_names_out())
X_test=pd.DataFrame(preprocessor.transform(X_test),columns=preprocessor.get_feature_names_out())

In [16]:
X_train

,num_pipeline__carat,num_pipeline__depth,num_pipeline__table,num_pipeline__x,num_pipeline__y,num_pipeline__z,cat_pipeline__cut,cat_pipeline__color,cat_pipeline__clarity
0,-0.975439,-0.849607,-0.121531,-1.042757,-1.080970,-1.123150,2.0,6.0,5.0
1,0.235195,1.833637,-0.121531,0.318447,0.279859,0.485354,3.0,3.0,3.0
2,0.494617,0.815855,0.399800,0.570855,0.606458,0.673737,0.0,2.0,2.0
3,-1.018676,0.260701,0.921131,-1.214034,-1.244270,-1.195605,0.0,2.0,0.0
4,-0.953821,-0.664555,-0.642862,-1.069801,-1.044681,-1.094168,2.0,1.0,5.0
...,...,...,...,...,...,...,...,...,...
135496,-1.040295,-0.016876,-0.642862,-1.268122,-1.244270,-1.239078,2.0,3.0,3.0
135497,0.991842,0.168176,-0.642862,1.048629,1.114501,1.079486,2.0,2.0,1.0
135498,0.451380,1.556060,-0.642862,0.516768,0.588314,0.702719,3.0,2.0,3.0
135499,0.667565,-1.774863,1.442462,0.868337,0.951202,0.688228,2.0,2.0,2.0


### Model Training

In [17]:
from sklearn.linear_model import LinearRegression,Ridge,Lasso,ElasticNet
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

In [18]:
def evaluate_model(test,predict):
    r2_value=r2_score(test,predict)
    mae=mean_absolute_error(test,predict)
    mse=mean_squared_error(test,predict)
    rmse=np.sqrt(mse)

    return r2_value,mae,mse,rmse

In [19]:
models={
    'LinerRegression': LinearRegression(),
    'Ridge':Ridge(),
    'Lasso':Lasso(),
    'Elastic Net':ElasticNet()
}

model_lst,r2_list=[],[]

for i in range(len(models)):
    model=list(models.values())[i]
    model.fit(X_train,y_train)

    y_pred=model.predict(X_test)

    r2,mae,mse,rmse=evaluate_model(y_test,y_pred)

    model_lst.append(list(models.keys())[i])
    r2_list.append(r2)

    print(list(models.keys())[i])
    print('R2 Value:',r2)
    print('mean_absolute_error:',mae)
    print('mean_squared_error:',mse)
    print('root_mean_squared_error:',rmse)
    print('\n')

print(model_lst,r2_list)

LinerRegression
R2 Value: 0.9117706806392406
mean_absolute_error: 726.0528982162476
mean_squared_error: 1437191.7172650357
root_mean_squared_error: 1198.8293111469354


Ridge
R2 Value: 0.9117705045154105
mean_absolute_error: 726.0819590791853
mean_squared_error: 1437194.5861946798
root_mean_squared_error: 1198.8305077010177




Lasso
R2 Value: 0.9117876827281106
mean_absolute_error: 727.5667075725919
mean_squared_error: 1436914.7655501505
root_mean_squared_error: 1198.713796345963


Elastic Net
R2 Value: 0.8369857410695969
mean_absolute_error: 1133.4879105147706
mean_squared_error: 2655384.2240686286
root_mean_squared_error: 1629.5349717231074


['LinerRegression', 'Ridge', 'Lasso', 'Elastic Net'] [0.9117706806392406, 0.9117705045154105, 0.9117876827281106, 0.8369857410695969]
